#PROBLEM 4: Knowledge Base Question Answering
Given is knowledge graph with entities and relations, questions with starting entity and answers, and their word embedding . For each question, navigate the graph from the start entity outwards until you find appropriate answer entities.<br>
Utils functions (similarity, load_graphs) are given, but you can choose not to use them. This python file contains the helper functions for this homework, the only update needed to use this file is to fill in the file paths.<br>

- The number of correct answers varies (could be 1, could many), use F1 to compare your answers with the given correct answers
- Utils functions (similarity, load_graphs) are given, but you can choose not to use them.
- Answers are given to be used for evaluation only, DO NOT USE ANSWERS IN YOUR GRAPH TRAVERSAL.
Your strategy should be a graph traversal augmented with scoring of paths; you might discard paths not promising along the way. This is similar to a focused crawl strategy. You will take a query (question) that you are trying to answer, it will have a starting entity. Begin your traversal at that starting entity, and look at all adjacent edges. Use get_rel_score_word2vecbase to get a similarity score for each edge, and traverse the edges that are promising. This part is up to you, you can cut off scores below a certain threshold, or take only the top percentage, or weight it based on the average.

There are many valid strategies. You will continue to traverse a path, until the score starts to decrease, or you notice the similarity score drops significantly (compared to the previous edges). Overall try a few different approaches, and choose one that gives you the best overall F1 score.

In [ ]:
import numpy as np
import pandas as pd
import requests
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from collections import Counter
import ast
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

In [154]:
file_path = 'https://www.khoury.northeastern.edu/home/vip/teach/DMcourse/6_graph_analysis/HW6/graph.txt'
response = requests.get(file_path)
lines = response.text.splitlines()
rows = []
for line in lines:
  line = line.strip()[1:-1].replace("'", "").split(", ")
  rows.append(line)

    # Convert the list of rows to a DataFrame with specified column names
graph_df = pd.DataFrame(rows, columns=['start_node', 'relation', 'end_node'])

In [155]:
graph_df.head()

,start_node,relation,end_node
0,m.01_d4,location.citytown.postal_codes,m.07nqmhw
1,m.09c7w0,base.locations.countries.states_provinces_within,m.06yxd
2,m.0fv_t,base.biblioness.bibs_location.state,m.06yxd
3,m.01_d4,location.citytown.postal_codes,m.07nqmhf
4,m.04yz499,military.military_command.military_conflict,m.014pd3


In [156]:
graph_df.nunique()

,0
start_node,286
relation,120
end_node,297


In [157]:
graph_df.shape

(1761, 3)

##Score of paths function

In [158]:
# Function to get the cosine similarity between a relation and query
# Note: Update this string with the path to the file
word2vec_model = models.Word2Vec.load('https://www.khoury.northeastern.edu/home/vip/teach/DMcourse/6_graph_analysis/HW6/word2vec_train_dev.dat')


def get_rel_score_word2vecbase(rel: str, query: str) -> float:
    """
    Get score for query and relation. Used to inform exploration of knowledge graph.

    :param rel: relation, or edge in knowledge graph
    :param query: query, question to answer
    :return: float score similarity between question and relation
    """
    # Relation not in embedding vocabulary
    if rel not in word2vec_model.wv:
        return 0.0
    # Relation must start with ns:
    rel = 'ns:' + rel if not rel[:3] == 'ns:' else rel
    words = word_tokenize(query.lower())
    w_embs = []
    for w in words:
        if w in word2vec_model.wv:
            w_embs.append(word2vec_model.wv[w])
    return np.mean(cosine_similarity(w_embs, [word2vec_model.wv[rel]]))

In [57]:
# This is a multihop query

# Starting node: 'm.09c7w0'
# Answer nodes: 'm.015smg', 'm.03q9wp2', 'm.03qtd_n', 'm.03qtf10'

query = 'what are major exports of the usa'
path1 = 'ns:location.statistical_region.major_exports'
path2 = 'ns:location.imports_exports_by_industry.industry'

# ['m.09c7w0', 'location.statistical_region.major_exports', 'm.04g4s8k']
# ['m.04g4s8k', 'location.imports_exports_by_industry.industry', 'm.03qtd_n']
# ['m.04g4s8k', 'location.imports_exports_by_industry.industry', 'm.03qtf10']
# ['m.04g4s8k', 'location.imports_exports_by_industry.industry', 'm.015smg']

# ['m.09c7w0', 'location.statistical_region.major_exports', 'm.04g4s90']
# ['m.04g4s90', 'location.imports_exports_by_industry.industry', 'm.03q9wp2']

In [58]:
get_rel_score_word2vecbase(path1,query)

0.30862752

In [59]:
get_rel_score_word2vecbase(path2,query)

0.30828074

In [159]:
def read_annotations(url):
  response = requests.get(url)
  lines = response.text.splitlines()
  data = []
  for line in lines:
    parsed_line = ast.literal_eval(line.strip()) # converts string to python data structure
    entry = {
        'id': parsed_line[0],
        'question': parsed_line[1],
        'starting_node': parsed_line[2],
        'paths': parsed_line[3],
        'meta': parsed_line[4],
        'answers': parsed_line[5]}


    data.append(entry)

  return data

# Example usage
url = 'https://www.khoury.northeastern.edu/home/vip/teach/DMcourse/6_graph_analysis/HW6/annotations.txt'
annotations = read_annotations(url)

In [98]:
# 55 index missing from annotation
len(annotations)

56

In [100]:
annotations[1]

{'id': 2,
 'question': 'what are major exports of the usa',
 'starting_node': 'm.09c7w0',
 'paths': [['ns:m.09c7w0',
   'ns:location.statistical_region.major_exports',
   '?y'],
  ['?y', 'ns:location.imports_exports_by_industry.industry', '?x']],
 'meta': 'United States of America',
 'answers': [{'AnswerType': 'Entity',
   'AnswerArgument': 'm.015smg',
   'EntityName': 'Automotive industry'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.03q9wp2',
   'EntityName': 'Food Manufacturing'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.03qtd_n',
   'EntityName': 'Pharmaceutical Preparation'},
  {'AnswerType': 'Entity',
   'AnswerArgument': 'm.03qtf10',
   'EntityName': 'Industrial Organic Chemicals, NEC'}]}

In [160]:
paths_length = []
paths = []
for i in range(len(annotations)):
  paths_length.append(len(annotations[i]['paths']))
  paths.append(annotations[i]['paths'])
print(dict(Counter(paths_length)))

# All answers are within 2-hops

{1: 36, 2: 20}


In [105]:
for idx,path in enumerate(paths):
  print(idx+1,path)

# Another observation is that there is only single relation per hop
# That particular relation might lead to multiple answers but there's only one relation per hop

1 [['ns:m.09c7w0', 'ns:location.location.time_zones', '?x']]
2 [['ns:m.09c7w0', 'ns:location.statistical_region.major_exports', '?y'], ['?y', 'ns:location.imports_exports_by_industry.industry', '?x']]
3 [['ns:m.07b_l', 'ns:location.location.time_zones', '?x']]
4 [['ns:m.034rd', 'ns:military.military_commander.military_commands', '?y'], ['?y', 'ns:military.military_command.military_conflict', '?x']]
5 [['ns:m.09c7w0', 'ns:location.statistical_region.religions', '?y'], ['?y', 'ns:location.religion_percentage.religion', '?x']]
6 [['ns:m.09889g', 'ns:people.person.children', '?x']]
7 [['ns:m.054c1', 'ns:sports.drafted_athlete.drafted', '?y'], ['?y', 'ns:sports.sports_league_draft_pick.draft', '?x']]
8 [['ns:m.09c7w0', 'ns:location.country.form_of_government', '?x']]
9 [['ns:m.034rd', 'ns:people.person.nationality', '?x']]
10 [['ns:m.09c7w0', 'ns:location.statistical_region.major_imports', '?y'], ['?y', 'ns:location.imports_exports_by_industry.industry', '?x']]
11 [['ns:m.09c7w0', 'ns:base.

#####Another observation: Knowledge of knowledge graph is incomplete

In [106]:
# For this question only 1 answer entity is in the graph: m.01zmpg

# [
#   22,
#   "who were michael jackson 's brothers and sisters",
#   'm.09889g',
#   [
#     [
#       'ns:m.09889g',
#       'ns:people.person.sibling_s',
#       '?y'
#     ],
#     [
#       '?y',
#       'ns:people.sibling_relationship.sibling',
#       '?x'
#     ]
#   ],
#   'Michael Jackson',
#   [
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.013v5j',
#       'EntityName': 'La Toya Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.01s993',
#       'EntityName': 'Jackie Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.01s9b9',
#       'EntityName': 'Tito Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.01zmpg',
#       'EntityName': 'Jermaine Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.023_5s',
#       'EntityName': 'Marlon Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.0247h4',
#       'EntityName': 'Rebbie Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.04_70v',
#       'EntityName': 'Randy Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.0598rr3',
#       'EntityName': "Joh'Vonnie Jackson"
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.0gbwp',
#       'EntityName': 'Janet Jackson'
#     },
#     {
#       'AnswerType': 'Entity',
#       'AnswerArgument': 'm.0nf3l7y',
#       'EntityName': 'Brandon Jackson'
#     }
#   ]
# ]

##Adjacency Matrix

In [161]:
unique_start_nodes = graph_df['start_node'].unique()
unique_end_nodes = graph_df['end_node'].unique()

# Combine unique nodes into a single list and get unique values
unique_nodes = set(unique_start_nodes).union(set(unique_end_nodes))

In [162]:
start_node = 'm.01_d4'
end_node = 'm.07nqmhw'

res = graph_df[(graph_df['start_node'] == start_node) & (graph_df['end_node'] == end_node)]['relation'].values

In [170]:
res[0]

'location.postal_code.country'

In [175]:
# Convert unique_nodes to a sorted list to ensure consistent ordering
unique_nodes_list = sorted(unique_nodes)

# Step 2: Create an empty adjacency matrix
adjacency_matrix = pd.DataFrame(
    '',
    index=unique_nodes_list,
    columns=unique_nodes_list
)

# Step 3: Populate the adjacency matrix based on edges
for _, row in graph_df.iterrows():
    start_node = row['start_node']
    end_node = row['end_node']
    res = graph_df[(graph_df['start_node'] == start_node) & (graph_df['end_node'] == end_node)]['relation'].values
    if len(res) > 0:
      adjacency_matrix.at[start_node, end_node] = res[0]


In [177]:
adjacency_matrix.shape

(297, 297)

#####1-hop Q where ans is at 3rd pos 'm.034rd' 'what country was george washington from'

In [218]:
query = 'what country was george washington from'
node = 'm.034rd'

query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
rel_score_df = rel_score_df.rename("rel_score")
query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
query_rel_df

,m.034rd,rel_score
m.04yy5qr,military.military_commander.military_commands,0.345896
m.09hxlzp,military.military_commander.military_commands,0.345896
m.049xrhp,military.military_commander.military_commands,0.345896
m.049yn2t,military.military_commander.military_commands,0.345896
m.04fvgyq,military.military_commander.military_commands,0.345896
m.04h_fs5,military.military_commander.military_commands,0.345896
m.04yv_tf,military.military_commander.military_commands,0.345896
m.04yv_x3,military.military_commander.military_commands,0.345896
m.04yvvkf,military.military_commander.military_commands,0.345896
m.04yxtkk,military.military_commander.military_commands,0.345896


#####1-hop Q where ans is at 1st pos 'm.09c7w0' 'what time zones are there in the us'

In [223]:
query = 'what time zones are there in the us'
node = 'm.09c7w0'

query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
rel_score_df = rel_score_df.rename("rel_score")
query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
query_rel_df

,m.09c7w0,rel_score
m.02lcrv,location.location.time_zones,0.349533
m.027wj2_,location.location.time_zones,0.349533
m.02lctm,location.location.time_zones,0.349533
m.02lcqs,location.location.time_zones,0.349533
m.02hczc,location.location.time_zones,0.349533
m.02hcv8,location.location.time_zones,0.349533
m.027wjl3,location.location.time_zones,0.349533
m.02fqwt,location.location.time_zones,0.349533
m.042g7t,location.location.time_zones,0.349533
m.09nqf,location.country.currency_used,0.260184


In [220]:
# If I try to do 2-hop
query = 'what time zones are there in the us'
node = 'm.02lcrv'

query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
rel_score_df = rel_score_df.rename("rel_score")
query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
query_rel_df

,m.02lcrv,rel_score
m.059g4,time.time_zone.locations_in_this_time_zone,0.298375
m.09c7w0,time.time_zone.locations_in_this_time_zone,0.298375


In [212]:
0.349533-0.298375

0.05115799999999998

#####2-hop Q where ans is at 1st pos 'm.09c7w0' 'what are major exports of the usa'

In [221]:
query = 'what are major exports of the usa'
node = 'm.09c7w0'

query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
rel_score_df = rel_score_df.rename("rel_score")
query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
query_rel_df

,m.09c7w0,rel_score
m.04g4s8q,location.statistical_region.major_exports,0.308628
m.04g4s8k,location.statistical_region.major_exports,0.308628
m.04g4s90,location.statistical_region.major_exports,0.308628
m.04g4s8w,location.statistical_region.major_exports,0.308628
m.03q1lwl,location.statistical_region.religions,0.294600
m.03q1lw4,location.statistical_region.religions,0.294600
m.03q1lvy,location.statistical_region.religions,0.294600
m.0_hhjkj,location.statistical_region.religions,0.294600
m.03q1lvq,location.statistical_region.religions,0.294600
m.03q1lvh,location.statistical_region.religions,0.294600


In [222]:
query = 'what are major exports of the usa'
node = 'm.04g4s8k'

query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
rel_score_df = rel_score_df.rename("rel_score")
query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
query_rel_df

,m.04g4s8k,rel_score
m.015smg,location.imports_exports_by_industry.industry,0.308281
m.03q9wp2,location.imports_exports_by_industry.industry,0.308281
m.03qtd_n,location.imports_exports_by_industry.industry,0.308281
m.03qtf10,location.imports_exports_by_industry.industry,0.308281


In [213]:
0.308628-0.308281

0.0003469999999999862

##BFS

In [270]:
def bfs(start_node,query,adjacency_matrix):
  queue = [start_node]
  answer_entities = set()
  global_max_rel_score = 0
  while len(queue)>0:
    node = queue.pop(0)
    query_df = adjacency_matrix.loc[node][adjacency_matrix.loc[node] != '']
    rel_score_df = query_df.apply(lambda rel: get_rel_score_word2vecbase('ns:'+rel, query))
    rel_score_df = rel_score_df.rename("rel_score")
    query_rel_df = pd.concat([query_df, rel_score_df], axis=1)
    query_rel_df.sort_values(by='rel_score', ascending=False,inplace=True)
    max_rel_score = query_rel_df['rel_score'].max()

    if max_rel_score > global_max_rel_score or abs(max_rel_score - global_max_rel_score)<0.001:
      global_max_rel_score = max_rel_score
      candidate_entities = set(query_rel_df[query_rel_df['rel_score'] == max_rel_score].index.tolist())
      queue.extend(list(candidate_entities))
      answer_entities = set()
      answer_entities.update(candidate_entities)
  return list(answer_entities)

In [249]:
bfs(annotations[0]['starting_node'],annotations[0]['question'],adjacency_matrix)

['m.02lcqs',
 'm.027wjl3',
 'm.02lcrv',
 'm.02fqwt',
 'm.027wj2_',
 'm.02lctm',
 'm.02hcv8',
 'm.02hczc',
 'm.042g7t']

In [241]:
[ans['AnswerArgument'] for ans in annotations[0]['answers']]

['m.027wj2_',
 'm.027wjl3',
 'm.02fqwt',
 'm.02hcv8',
 'm.02hczc',
 'm.02lcqs',
 'm.02lcrv',
 'm.02lctm',
 'm.042g7t']

In [269]:
annotations[1]['starting_node'],annotations[1]['question']

('m.09c7w0', 'what are major exports of the usa')

##F-1 Score

In [ ]:
# If Actual Ans is following set of entities: [E1, E2, E3]
# And algorithm's ans is: [E2, E3, E4]
# TP = 2 (E2,E3) ; FN = 1 (E1) ; FP = 1 (E4)
# Precision = TP/(TP+FP) ; Recall = TP/(TP+FN)
# F1 = Harmonic_Mean(Precision, Recall)

In [265]:
y_true = [['E1','E2','E3']]
y_pred = [['E2','E3','E4']]

binarizer = MultiLabelBinarizer()
binarizer.fit(y_true)

f1_score(binarizer.transform(y_true),
         binarizer.transform(y_pred),
         average='macro')

0.6666666666666666

In [273]:
sum_f1_score = 0

for i in tqdm(range(len(annotations))):
  y_true = [[ans['AnswerArgument'] for ans in annotations[i]['answers']]]
  y_pred = [bfs(annotations[i]['starting_node'],annotations[i]['question'],adjacency_matrix)]

  binarizer = MultiLabelBinarizer()
  binarizer.fit(y_true)

  sum_f1_score += f1_score(binarizer.transform(y_true), binarizer.transform(y_pred),average='macro')

print(f"\nAverage F1 score: {sum_f1_score/len(annotations)}")

100%|██████████| 56/56 [00:13<00:00,  4.02it/s]


Average F1 score: 0.46540458873906515
